In [1]:
#import libraries for pdf reading, spacy, pandas
import io
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage

import spacy
import pandas as pd
import spacy
nlp = spacy.load('en_core_web_md')

In [2]:
# create function for reading pdf into a list of pages
def extract_text_by_page(pdf_path):
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      check_extractable=True):
            resource_manager = PDFResourceManager()
            fake_file_handle = io.StringIO()
            converter = TextConverter(resource_manager, fake_file_handle)
            page_interpreter = PDFPageInterpreter(resource_manager, converter)
            page_interpreter.process_page(page)
            text = fake_file_handle.getvalue()
            yield text
            # close open handles
            converter.close()
            fake_file_handle.close()
def extract_text(pdf_path):
    content=[]
    for page in extract_text_by_page(pdf_path):
        content.append(page)
    return content

In [5]:
#use function to read Microsoft's transcripts pdf
txt1 = extract_text('Transcript batch1 2019.pdf')

In [9]:
df1 = pd.DataFrame(txt1, columns = ['text'])

df1.to_csv('Transcript batch1 2019.csv')

In [3]:
txt2 = extract_text('Transcript batch2 2019.pdf')

In [12]:
df2 = pd.DataFrame(txt2, columns = ['text'])

df2.to_csv('Transcript batch2 2019.csv')

In [11]:
txt3 = extract_text('Transcript batch3 2019.pdf')

In [13]:
df3 = pd.DataFrame(txt3, columns = ['text'])

df3.to_csv('Transcript batch3 2019.csv')

In [14]:
txt4 = extract_text('Transcript batch4 2019.pdf')

In [15]:
df4 = pd.DataFrame(txt4, columns = ['text'])

df4.to_csv('Transcript batch4 2019.csv')

In [16]:
pdlist = [df1, df2, df3, df4]
dft = pd.concat(pdlist)

In [131]:
dft.shape

(12907, 10)

In [18]:
txt5 = extract_text('Transcript batch5 2019.pdf')

In [19]:
df5 = pd.DataFrame(txt5, columns = ['text'])

df5.to_csv('Transcript batch5 2019.csv')

In [135]:
pdlist = [df1, df2, df3, df4, df5]

In [136]:
#convert list to dataframe
df = pd.concat(pdlist)

df.head()

,text
0,COPYRIGHT © 2019 S&P Global Market Intelligen...
1,ContentsCOPYRIGHT © 2019 S&P Global Market Int...
2,"BANCO BRADESCO S.A. SPECIAL CALL | MAY 06, 20..."
3,"BANCO BRADESCO S.A. SPECIAL CALL | MAY 06, 20..."
4,"BANCO BRADESCO S.A. SPECIAL CALL | MAY 06, 20..."


In [140]:
#split page by sentences, list of strings
df['txt_list'] = None

for i in range(len(df['text'])):
    txt_list = []
    for sent in nlp(df['text'].iloc[i]).sents:
        txt_list.append(sent)
    df['txt_list'].iloc[i] = txt_list

In [259]:
#use first sentence as a heading
df['heading'] = None

for i in range(len(df['txt_list'])):
    df['heading'].iloc[i] = str(df['txt_list'].iloc[i][0])

/Users/ahmedalali/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [277]:
#tage documents
df['match'] = None
for i in range(len(df['heading'])):
    if ((df['heading'].iloc[i] == df['heading'].iloc[i-1]) or (df['heading'].iloc[i] == df['heading'].iloc[i+1])) and ('|' in df['text'].iloc[i]):
        df['match'].iloc[i] = True
    else:
        df['match'].iloc[i] = False

In [273]:
df.head()

,text,doc,dates,txt_list,heading,match,company,date
0,COPYRIGHT © 2019 S&P Global Market Intelligen...,other,None,"[( , COPYRIGHT, ©, 2019, S&P, Global, Market, ...",COPYRIGHT © 2019 S&P Global Market Intelligen...,False,"(BANCO, BRADESCO)",None
1,ContentsCOPYRIGHT © 2019 S&P Global Market Int...,other,None,"[(ContentsCOPYRIGHT, ©, 2019, S&P, Global, Mar...",ContentsCOPYRIGHT © 2019 S&P Global Market Int...,False,"(BANCO, BRADESCO)",None
2,"BANCO BRADESCO S.A. SPECIAL CALL | MAY 06, 20...",other,"MAY 06, 2019","[(BANCO, BRADESCO, S.A., SPECIAL, CALL, |, ),...",BANCO BRADESCO S.A. SPECIAL CALL |,True,"(BANCO, BRADESCO)",None
3,"BANCO BRADESCO S.A. SPECIAL CALL | MAY 06, 20...",other,"MAY 06, 2019","[(BANCO, BRADESCO, S.A., SPECIAL, CALL, |, ),...",BANCO BRADESCO S.A. SPECIAL CALL |,True,"(BANCO, BRADESCO)",None
4,"BANCO BRADESCO S.A. SPECIAL CALL | MAY 06, 20...",other,"MAY 06, 2019","[(BANCO, BRADESCO, S.A., SPECIAL, CALL, |, ),...",BANCO BRADESCO S.A. SPECIAL CALL |,True,"(BANCO, BRADESCO)",None


### Restart here

In [222]:
#label documents for earnings calls and extract dates
df['doc'] = None
for i in range(len(df['text'])):
    if ('Earnings Call Transcripts' in df['text'].iloc[i]) or ('EARNINGS CALL |' in df['text'].iloc[i]):
        df['doc'].iloc[i] = 'earnings call'
    else:
        df['doc'].iloc[i] = 'other'

/Users/ahmedalali/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [309]:
#for i in range(len(df['doc'])):
#    if (df['doc'].iloc[i] == 'other') and (df['doc'].iloc[i-1] == 'earnings call') and (df['doc'].iloc[i+1] == 'earnings call'):
#        df['doc'].iloc[i] = 'earnings call'

/Users/ahmedalali/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [315]:
df['text'].iloc[3470][11:14]

'FH2'

In [316]:
df['dates'] = None

for i in range(len(df['text'])):
    if df['match'].iloc[i] == True: 
        x = df['text'].iloc[i].split()
        #print(i)
        df['dates'].iloc[i] = ' '.join(x[x.index('|')+1:x.index('|')+4])[:-9]

/Users/ahmedalali/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [317]:
#print(df['text'].iloc[3478])
#print(df['text'].iloc[3477])
#print(str(df['heading'].iloc[866]) == str(df['heading'].iloc[865]))
df.iloc[3470:3485]

,text,doc,dates,txt_list,heading,match,company,date
3470,"AEGON N.V. FH2 2018 EARNINGS CALL | FEB 14, 2...",earnings call,"FEB 14, 2019","[(AEGON, N.V., FH2, 2018, EARNINGS, CALL, |, ...","AEGON N.V. FH2 2018 EARNINGS CALL | FEB 14,",True,"(AEGON, N.V.)","[February, 14, 2019]"
3471,"AEGON N.V. FH2 2018 EARNINGS CALL | FEB 14, 2...",earnings call,"FEB 14, 2019","[(AEGON, N.V., FH2, 2018, EARNINGS, CALL, |, ...","AEGON N.V. FH2 2018 EARNINGS CALL | FEB 14,",True,"(AEGON, N.V.)","[February, 14, 2019]"
3472,"AEGON N.V. FH2 2018 EARNINGS CALL | FEB 14, 2...",earnings call,"FEB 14, 2019","[(AEGON, N.V., FH2, 2018, EARNINGS, CALL, |, ...","AEGON N.V. FH2 2018 EARNINGS CALL | FEB 14,",True,"(AEGON, N.V.)","[February, 14, 2019]"
3473,"AEGON N.V. FH2 2018 EARNINGS CALL | FEB 14, 2...",earnings call,"FEB 14, 2019","[(AEGON, N.V., FH2, 2018, EARNINGS, CALL, |, ...","AEGON N.V. FH2 2018 EARNINGS CALL | FEB 14,",True,"(AEGON, N.V.)","[February, 14, 2019]"
3474,"AEGON N.V. FH2 2018 EARNINGS CALL | FEB 14, 2...",earnings call,"FEB 14, 2019","[(AEGON, N.V., FH2, 2018, EARNINGS, CALL, |, ...","AEGON N.V. FH2 2018 EARNINGS CALL | FEB 14,",True,"(AEGON, N.V.)","[February, 14, 2019]"
3475,COPYRIGHT © 2019 S&P Global Market Intelligen...,earnings call,None,"[( , COPYRIGHT, ©, 2019, S&P, Global, Market, ...",COPYRIGHT © 2019 S&P Global Market Intelligen...,False,"(ContentsCOPYRIGHT, ©)","[February, 14, 2019]"
3476,ContentsCOPYRIGHT © 2019 S&P Global Market Int...,other,None,"[(ContentsCOPYRIGHT, ©, 2019, S&P, Global, Mar...",ContentsCOPYRIGHT © 2019 S&P Global Market Int...,False,"(ContentsCOPYRIGHT, ©)","[February, 14, 2019]"
3477,ContentsCOPYRIGHT © 2019 S&P Global Market Int...,other,None,"[(ContentsCOPYRIGHT, ©, 2019, S&P, Global, Mar...",ContentsCOPYRIGHT © 2019 S&P Global Market Int...,False,"(ContentsCOPYRIGHT, ©)","[February, 14, 2019]"
3478,"AMERICAN INTERNATIONAL GROUP, INC. FQ4 2018 EA...",earnings call,"FEB 14, 2019","[(AMERICAN, INTERNATIONAL, GROUP, ,), (INC, .,...","AMERICAN INTERNATIONAL GROUP,",True,"(AMERICAN, INTERNATIONAL)","[February, 14, 2019]"
3479,"AMERICAN INTERNATIONAL GROUP, INC. FQ4 2018 EA...",earnings call,"FEB 14, 2019","[(AMERICAN, INTERNATIONAL, GROUP, ,), (INC, .,...","AMERICAN INTERNATIONAL GROUP,",True,"(AMERICAN, INTERNATIONAL)","[February, 14, 2019]"


In [318]:
for i in range(len(df['dates'])):
    if df['match'].iloc[i] == False:
        df['dates'].iloc[i] = df['dates'].iloc[i+3]

/Users/ahmedalali/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [319]:
df[df['dates'].isnull()]

,text,doc,dates,txt_list,heading,match,company,date


In [320]:
df[df['dates'] == 'NaT']

,text,doc,dates,txt_list,heading,match,company,date


In [321]:
x = df['text'].iloc[669].split()

' '.join(x[x.index('|')+1:x.index('|')+4])#[:-9]

'MAY16, 2019Copyright ©'

In [332]:
for i in range(len(df['dates'])):
    df['dates'].iloc[i] = pd.to_datetime(df['dates'].iloc[i], errors = 'ignore')

/Users/ahmedalali/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [323]:
type(df['dates'].iloc[667])

str

In [324]:
df[df['dates'].apply(lambda x: isinstance(x, pd._libs.tslibs.nattype.NaTType))]

,text,doc,dates,txt_list,heading,match,company,date


In [338]:
df.head()

,text,doc,dates,txt_list,heading,match,company,date
0,COPYRIGHT © 2019 S&P Global Market Intelligen...,other,2019-05-06 00:00:00,"[( , COPYRIGHT, ©, 2019, S&P, Global, Market, ...",COPYRIGHT © 2019 S&P Global Market Intelligen...,False,None,None
1,ContentsCOPYRIGHT © 2019 S&P Global Market Int...,other,2019-05-06 00:00:00,"[(ContentsCOPYRIGHT, ©, 2019, S&P, Global, Mar...",ContentsCOPYRIGHT © 2019 S&P Global Market Int...,False,None,None
2,"BANCO BRADESCO S.A. SPECIAL CALL | MAY 06, 20...",other,2019-05-06 00:00:00,"[(BANCO, BRADESCO, S.A., SPECIAL, CALL, |, ),...",BANCO BRADESCO S.A. SPECIAL CALL |,True,BANCO BRADESCO,None
3,"BANCO BRADESCO S.A. SPECIAL CALL | MAY 06, 20...",other,2019-05-06 00:00:00,"[(BANCO, BRADESCO, S.A., SPECIAL, CALL, |, ),...",BANCO BRADESCO S.A. SPECIAL CALL |,True,BANCO BRADESCO,None
4,"BANCO BRADESCO S.A. SPECIAL CALL | MAY 06, 20...",other,2019-05-06 00:00:00,"[(BANCO, BRADESCO, S.A., SPECIAL, CALL, |, ),...",BANCO BRADESCO S.A. SPECIAL CALL |,True,BANCO BRADESCO,None


In [337]:
# exctract compaby name, first 2 words only
df['company'] = None

for i in range(len(df['heading'])): 
    if df['match'].iloc[i] == True:
        df['company'].iloc[i] = ' '.join(df['text'].iloc[i].split()[:2])

/Users/ahmedalali/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [339]:
for i in range(len(df['company'])):
    if (df['match'].iloc[i] == False) and (df['company'].iloc[i] == None):
        df['company'].iloc[i] = df['company'].iloc[i+3]

/Users/ahmedalali/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [374]:
dfe = df[df['doc'] == 'earnings call']

In [388]:
dff = dfe.groupby(['company', 'dates']).agg({'company' : 'first', 'dates' : 'first', 'text':'sum'})

In [389]:
dff.reset_index(inplace = True, drop=True)

In [390]:
dff.head(20)

,company,dates,text
0,3M COMPANY,2019-07-25 00:00:00,COPYRIGHT © 2019 S&P Global Market Intelligen...
1,A.P. MØLLER,2019-05-24 00:00:00,COPYRIGHT © 2019 S&P Global Market Intelligen...
2,A.P. MØLLER,2019-08-15 00:00:00,COPYRIGHT © 2019 S&P Global Market Intelligen...
3,AB VOLVO,2019-01-30 00:00:00,COPYRIGHT © 2019 S&P Global Market Intelligen...
4,AB VOLVO,2019-04-24 00:00:00,COPYRIGHT © 2019 S&P Global Market Intelligen...
5,AB VOLVO,2019-07-18 00:00:00,COPYRIGHT © 2019 S&P Global Market Intelligen...
6,ABB LTD,2019-02-28 00:00:00,COPYRIGHT © 2019 S&P Global Market Intelligen...
7,ABB LTD,2019-07-25 00:00:00,COPYRIGHT © 2019 S&P Global Market Intelligen...
8,ABBOTT LABORATORIES,2019-07-17 00:00:00,COPYRIGHT © 2019 S&P Global Market Intelligen...
9,ABBVIE INC.,2019-07-26 00:00:00,COPYRIGHT © 2019 S&P Global Market Intelligen...


In [391]:
dff.shape

(669, 3)

In [392]:
len(dff.company.unique())

313

In [393]:
dff.company.value_counts().mean()

2.137380191693291

In [394]:
dff.to_csv('G500_Earnings_2019.csv')

In [353]:
dff['dates'].iloc[2]

Timestamp('2019-08-15 00:00:00')

In [354]:
dff.columns

Index(['dates', 'text', 'company'], dtype='object')

In [382]:
dfx = pd.DataFrame(dff[ 'company'])

In [383]:
dfx.head()

company
company     dates                           
3M COMPANY  2019-07-25 00:00:00   3M COMPANY
A.P. MØLLER 2019-05-24 00:00:00  A.P. MØLLER
            2019-08-15 00:00:00  A.P. MØLLER
AB VOLVO    2019-01-30 00:00:00     AB VOLVO
            2019-04-24 00:00:00     AB VOLVO